In [1]:
import torch
import torch.nn as nn

In [2]:
inputs = torch.Tensor(1,1,28,28)

In [3]:
print(inputs.shape)

torch.Size([1, 1, 28, 28])


# 합성곱층의 풀링 선언하기 

In [5]:
# 1channerl을 입력 받아서 32로 만듭니다, kernel 사이즈는 3이고  딩은 1
conv1 = nn.Conv2d(1,32,3,padding=1)
conv2 = nn.Conv2d(32,64,3,padding=1)
pool = nn.MaxPool2d(2)

In [10]:
out = conv1(inputs)
print(out.shape)
out = conv2(out)
print(out.shape)
out = pool(out)
print(out.shape)
out = pool(out)
print(out.shape)

torch.Size([1, 32, 28, 28])
torch.Size([1, 64, 28, 28])
torch.Size([1, 64, 14, 14])
torch.Size([1, 64, 7, 7])


In [11]:
# 첫번째 차원인 배치 차원은 그대로 두고 나머지는 펼쳐라 
out = out.view(out.size(0),-1)
print(out.shape)

torch.Size([1, 3136])


In [12]:
fc = nn.Linear(3136,10)
out = fc(out)

In [14]:
out.shape

torch.Size([1, 10])

In [15]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [ ]:
device = 'cuda'
torch.cuda.set_device